Import packages and load rotation functions

In [47]:
# import the necessary packages
#Life is good
from PIL import Image
import pytesseract
import numpy as np
#from pytesseract import image_to_string
#from pytesseract import image_to_boxes
#from pytesseract import image_to_data
import argparse
import cv2
import os

def orientationScore(image,skipStep = 10):
    totValue = 0
    for row in range(len(image)):
        rowVal = 0
        for col in range(0,len(image[0]),skipStep):
            rowVal += image[row][col]
        totValue += rowVal*rowVal
    return totValue

def findOrient(image,angleRange = 5,skipStep = 10):
    maxValue = -1
    maxRot  = 0
    imageRows, imageCols = image.shape
    accuracy = 0.1



    for i in range(-1, angleRange*2*int(1/accuracy)):
        #rot = i
        rot = (round(i / 2 - 0.4) + 1) * ((-1) * (i % 2) + ((i + 1) % 2)) * 0.1
        #print rot
        # print("Rotation: ",rot,". i: ",i)
        M = cv2.getRotationMatrix2D((imageCols / 2, imageRows / 2), rot, 1)
        dst = cv2.warpAffine(image, M, (imageCols, imageRows), borderValue=(255, 255, 255))

        value = orientationScore(255-dst,skipStep = 10)
        if value>maxValue:
            maxValue = value
            maxRot = rot
        #print "Current angle: ",rot,". Current value: ", value,". Max value: ",maxValue,". Max rot: ",maxRot
        #cv2.imshow("Rotated Image: ",dst)
        #cv2.waitKey(0)

    M = cv2.getRotationMatrix2D((imageCols / 2, imageRows / 2), maxRot, 1)
    dst = cv2.warpAffine(image, M, (imageCols, imageRows), borderValue=(255, 255, 255))
    #cv2.imshow("Rotated Image: ", dst)
    #cv2.waitKey(0)
    return dst

Specify the paramaters of the OCR run

In [55]:
args = {}
#args["image"] = "/home/dries/devel/PraelexisOCR/Code/rotate.png"
args["image"] = "/home/dries/devel/PraelexisOCR/Code/rotatedDistorted.png"
#args["image"] = "/home/dries/devel/PraelexisOCR/Code/RealScan.jpg"
#args["image"] = "/home/dries/devel/PraelexisOCR/Code/phonePhoto.jpg"
args["preprocess"] = ""
args["rotationSearch"] = 20

Main code

In [56]:

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,help="path to input image to be OCR'd")
# ap.add_argument("-p", "--preprocess", type=str, default="thresh",	help="type of preprocessing to be done")
# args = vars(ap.parse_args())s


# load the example image and convert it to grayscale
image = cv2.imread(args["image"])

# cv2.imshow("Image", image)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

print("Finding orientation..")
gray = findOrient(gray,angleRange = args["rotationSearch"],skipStep = 10)
print("Image corrected. Scanning image..")
# check to see if we should apply thresholding to preprocess the
# image
if args["preprocess"] == "thresh":
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# make a check to see if median blurring should be done to remove
# noise
elif args["preprocess"] == "blur":
    gray = cv2.medianBlur(gray, 1)

'''cv2.namedWindow('Input', cv2.WINDOW_NORMAL)
cv2.imshow("Input", gray)    
cv2.waitKey(0)'''



# write the grayscale image to disk as a temporary file so we can
# apply OCR to it
#filename = "{}.png".format(os.getpid())
#cv2.imwrite(filename, gray)

# load the image as a PIL/Pillow image, apply OCR, and then delete
# the temporary file
#text = pytesseract.image_to_string(gray,boxes=False)


# This might not be the best approach. I think the system also directly searches for words.
# These word phrases are not included in the config settings.
#con ="-c tessedit_char_whitelist=0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLNMOPQRSTUVWXYZ-+,.()&:_"

cv2.imwrite('PreProcessed.png',gray)

text = pytesseract.image_to_string(gray,boxes=False)#,config=con)
data = pytesseract.image_to_string(gray,boxes=True)#,config=con)

lines = data.split('\n')

print "Text: ", text

array = []


yLen = len(gray)
newPage = gray.copy()

for i in range(len(newPage)):
    for j in range(len(newPage[0])):
        newPage[i][j] = 255

for line in lines:
    entry = line.split(' ')
    #print(entry)
    array.append(entry)
    cv2.rectangle(gray, (int(entry[1]), yLen-int(entry[2])), (int(entry[3]), yLen-int(entry[4])), (0,0,0), 4)
    font = cv2.FONT_HERSHEY_SIMPLEX
    #print("Here: ",entry[0], " ", len(entry[0]))
    try:
        char = str(entry[0])
        #print(char)
        
        xSize = 0.5*(int(entry[3])-int(entry[1]))
        ySize = 0.5*(int(entry[4])-int(entry[2]))
        
        size = xSize
        
        if ySize>xSize:
            size = ySize
        
        fondSize = size/15.0
        loc = (int(entry[1])+int(0.5*(int(entry[3])-int(entry[1]))),yLen-int(entry[2])+int(0.5*(int(entry[4])-int(entry[2]))))
        cv2.putText(newPage,char,loc,font, fondSize,(0,0,255))
        print char," ",fondSize, " ", loc
    except (TypeError, ValueError):
        print "Not added: ",entry
        xSize = 0.5*(int(entry[3])-int(entry[1]))
        ySize = 0.5*(int(entry[4])-int(entry[2]))
        
        size = xSize
        
        if ySize>xSize:
            size = ySize
        
        fondSize = size/15.0
        loc = (int(entry[1])+int(0.5*(int(entry[3])-int(entry[1]))),yLen-int(entry[2])+int(0.5*(int(entry[4])-int(entry[2]))))
        cv2.putText(newPage,'???',loc,font, fondSize,(0,0,255))
        
        
#data = pytesseract.image_to_boxes(gray)
#os.remove(filename)
#print "Text found:\n",text

#print "Data:\n",array
# show the output images

#for entry in data:
    #print "Entry: ", entry
    #cv2.rectangle(gray, (100, 100), (1000, 1000), (0,0,0), 20)
#cv2.namedWindow('Input', cv2.WINDOW_NORMAL)
#cv2.imshow("Input", gray)
cv2.imwrite('Boxes.png',gray)
cv2.imwrite('Estimation.png',newPage)

Finding orientation..
Image corrected. Scanning image..
Text:  mwhmmmmmmmhmmm

lumxn.wm—.wmlwvdnuﬁnllwmmam-nmd
mumeMWMMammmMmm,.nmmp-MMm
mmu-mxmmmcmmmmmx. mmnmm
Mnemuwmmmmmam‘mlmmm. (Samara
mmnmm mewnnm') mumwnmmdumﬂapm
mmnmmu-mvumhmMp-mmnuumopmw
Mm: wm mluumn n: mu nu much um. um pan-mg. l mug:
muwwmvﬂmmMmMme:kﬂm
mmmmmmmummmwmmwmmmm
mwmylumumlauupy

InlmmlmnllnlnMnlwumhmmlmmmnmlmm
mvmmmhxpmlmb:
m   1.63333333333   (48, 47)
w   0.866666666667   (90, 47)
b   0.333333333333   (108, 46)
m   0.533333333333   (124, 46)
W   0.6   (145, 46)
W   1.56666666667   (180, 50)
U   0.333333333333   (213, 46)
M   1.03333333333   (236, 46)
M   0.333333333333   (260, 44)
E   0.566666666667   (277, 47)
M   0.866666666667   (303, 47)
I   0.233333333333   (321, 44)
m   0.5   (331, 46)
K   0.333333333333   (344, 46)
Not added:  [u'\u2018', u'348', u'282', u'351', u'284', u'0']
l   0.333333333333   (46, 66)
u   0.233333333333   (52, 64)
m   0.933333333333   (74, 66)
x   0.233333333333   (90, 64)
n   0.233333

True

In [ ]:
##pytesseract.image_to_string??